### ResNet Exercise:

In the file model_ResNetv2.py you can find an accurate implementation of the ResNet v2 (from here). Create a tf.Dataset containing CIFAR10, apply to it some basic image preprocessing (avoid rotation!) and train a ResNet56_v2 model on it for at least 30 epochs. Distribute the training over multiple GPUs (try at least with 2) and test different batch sizes. How does the time per epoch change? What about the validation accuracy?

#### Load Modules

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard, LearningRateScheduler

import matplotlib.pyplot as plt
import datetime
from day2.model_ResNetv2 import resnet_v2 as resnet

#### Parameters

In [2]:
batch=128
epochs=30 
height=32
width=32

#### Load & Preprocessing & DA

In [3]:
(x, y), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [4]:
print(x.shape, y.shape)

(50000, 32, 32, 3) (50000, 1)


In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((x, y))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [6]:
def augmentation(x, y):
    x = tf.image.resize_with_crop_or_pad(
        x, height + 8, width + 8)
    x = tf.image.random_crop(x, [height, width, 3])
    x = tf.image.random_flip_left_right(x)
    return x, y

def preprocess(x, y):
    x = tf.image.per_image_standardization(x)
    return x, y

In [7]:
train_dataset = train_dataset.map(augmentation)

In [8]:
train_dataset = (train_dataset
                 .map(augmentation)
                 .shuffle(buffer_size=50000))

In [9]:
def normalize(x, y):
    x = tf.image.per_image_standardization(x)
    return x, y

In [10]:
train_dataset = (train_dataset
                 .map(augmentation)
                 .shuffle(buffer_size=50000)
                 .map(normalize))

In [11]:
train_dataset = (train_dataset.map(augmentation)
                 .map(normalize)
                 .shuffle(50000)
                 .batch(128, drop_remainder=True))

test_dataset = test_dataset.map(preprocess).batch(128*2, drop_remainder=True)

In [12]:
# Input image dimensions.
input_shape = x.shape[1:]

In [13]:
#model = resnet(input_shape=input_shape , depth=56)

In [14]:
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    #model = resnet.resnet56(classes=10)
    model = resnet(input_shape=input_shape , depth=56)
    model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])  

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensor

In [19]:
LR_SCHEDULE = [(0.1, 30), (0.01, 45)]

def schedule(epoch):
    initial_learning_rate = 0.1 * 128 / 128
    learning_rate = initial_learning_rate
    for mult, start_epoch in LR_SCHEDULE:
        if epoch >= start_epoch:
            learning_rate = initial_learning_rate * mult
        else:
            break
    tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
    return learning_rate

In [21]:
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
file_writer.set_as_default()
tensorboard_callback = TensorBoard(
  log_dir=log_dir,
  update_freq='batch',
  histogram_freq=1)

lr_schedule_callback = LearningRateScheduler(schedule)

history = model.fit(train_dataset,
          epochs=30,
          validation_data=test_dataset,
          validation_freq=1,
          callbacks=[tensorboard_callback, lr_schedule_callback])
history = model.evaluate(test_dataset)

history = model.save('model_resnet_v2.h5')

Epoch 1/30
390/390 [==============================] - 24s 63ms/step - accuracy: 0.4026 - loss: 2.6115 - val_accuracy: 0.1694 - val_loss: 4.9742 - lr: 0.1000
Epoch 2/30
390/390 [==============================] - 20s 52ms/step - accuracy: 0.4832 - loss: 2.2907 - val_accuracy: 0.3947 - val_loss: 2.6633 - lr: 0.1000
Epoch 3/30
390/390 [==============================] - 20s 52ms/step - accuracy: 0.5315 - loss: 2.0660 - val_accuracy: 0.4272 - val_loss: 2.4211 - lr: 0.1000
Epoch 4/30
390/390 [==============================] - 20s 52ms/step - accuracy: 0.5684 - loss: 1.8869 - val_accuracy: 0.3982 - val_loss: 2.5601 - lr: 0.1000
Epoch 5/30
390/390 [==============================] - 20s 52ms/step - accuracy: 0.5987 - loss: 1.7284 - val_accuracy: 0.3976 - val_loss: 2.5777 - lr: 0.1000
Epoch 6/30
390/390 [==============================] - 20s 52ms/step - accuracy: 0.6234 - loss: 1.6008 - val_accuracy: 0.5130 - val_loss: 1.9399 - lr: 0.1000
Epoch 7/30
390/390 [==============================] - 20s 

In [22]:
history = model.evaluate(test_dataset)

39/39 [==============================] - 1s 21ms/step - accuracy: 0.6509 - loss: 1.3283
